In [1]:
# This file is used to extract the appropriate features
# and extract them to
    # gradient.npy
    # label.npy

In [2]:
# all import
import cv2
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
# get pos_list/neg_list from the folder you create before
pos_list = []
neg_list = []
filenames_pos = os.listdir(r'.\\starfish-image')
for i in filenames_pos:
    image_path = '.\\starfish-image\\' + i
    img = cv2.imread(image_path)
    original_img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pos_list.append(original_img_RGB)

filenames_neg = os.listdir(r'.\\neg-image')
for i in filenames_neg:
    image_path = '.\\neg-image\\' + i
    img = cv2.imread(image_path)
    original_img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    neg_list.append(original_img_RGB)

In [4]:
# get gradient image for single raw image
from skimage.feature import hog
def computeHOGs(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    final_img = cv2.resize(gray_img, (100, 100), interpolation=cv2.INTER_CUBIC)
    features, hog_img = hog(final_img, orientations=7, pixels_per_cell=(2, 2), cells_per_block=(1, 1), visualize=True,feature_vector=False)
    plt.imshow(hog_img, 'gray')
    return hog_img.flatten()
    # return gradient

In [5]:
# from skimage.feature import hog
def get_color_histo_feature(img):
    img = cv2.resize(img, (10, 10), interpolation=cv2.INTER_CUBIC)
    channel1_hist = np.histogram(img[:,:,0])
    channel2_hist = np.histogram(img[:,:,1])
    channel3_hist = np.histogram(img[:,:,2])
    return np.concatenate((channel1_hist[0],channel2_hist[0],channel3_hist[0]))

# def bin_spatial(img,size=(150,150)):
#     features = cv2.resize(img,size).ravel()
#     return features

# 拼接hog，color和spatial
def extractor(img):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    img_features = []
    img_features.append(computeHOGs(img))
    img_features.append(get_color_histo_feature(img))
    return np.concatenate(img_features)

In [ ]:
gradient = []
label = []
# use computeHOGs to only get the hog feature
# computeHOGs() get only hog
# extractor() to get the hog, color, spatial feature
for i in tqdm(range(len(pos_list))):
    gradient.append(extractor(pos_list[i]))
for i in tqdm(range(len(neg_list))):
    gradient.append(extractor(neg_list[i]))

gradient_list=np.array(gradient)
np.save('gradient.npy',gradient_list) #
# gradient = np.load('gradient.npy')
# gradient = gradient.tolist()

for _ in tqdm(range(len(pos_list))):
    label.append(1)
for _ in tqdm(range(len(neg_list))):
    label.append(-1)
label_list=np.array(label)
np.save('label.npy',label_list) #
# label = np.load('label.npy')
# label = label.tolist()